# IBM Code Generation and Optimization

**Nama:** Ahmad Kholish Fauzan Shobiry  
**Proyek:** *Nusantara Story*

In [ ]:
# --- Persiapan Lingkungan ---
print("--- Demostrasi Penggunaan IBM Granite untuk Code Generation ---")
print("Memasang library yang dibutuhkan...")

# Instal library yang Anda sebutkan
!pip install -q git+https://github.com/ibm-granite-community/utils \
    "langchain_community<0.3.0" \
    replicate

print("Library berhasil dipasang.")

--- Demostrasi Penggunaan IBM Granite untuk Code Generation ---
Memasang library yang dibutuhkan...
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 61.0/61.0 kB 4.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.3/2.3 MB 26.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 48.6/48.6 kB 3.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.0/1.0 MB 49.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 397.1/397.1 kB 23.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 311.8/311.8 kB 22.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 18.3/18.3 MB 79.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 50.9/50.9 kB 3.7 MB/s eta 0:00:00
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the sourc

In [ ]:
from ibm_granite_community.notebook_utils import get_env_var
from langchain_community.llms import Replicate
import os

# Ambil token dari Secrets Colab
REPLICATE_API_TOKEN = get_env_var('REPLICATE_API_TOKEN')

if not REPLICATE_API_TOKEN:
    print("Error: REPLICATE_API_TOKEN tidak ditemukan. Harap tambahkan di Secrets Colab dan pastikan 'Notebook access' diaktifkan.")
else:
    print("\nREPLICATE_API_TOKEN berhasil dimuat.")

    # Inisialisasi model Granite menggunakan Langchain Replicate
    # Model Granite adalah model Granite 3.3 8B Instruct yang Anda gunakan.
    # Konfigurasi model diambil dari Replicate Docs.
    model = Replicate(
        model="ibm-granite/granite-3.3-8b-instruct",
        replicate_api_token=REPLICATE_API_TOKEN,
        model_kwargs={"max_tokens": 10000, "temperature": 0.6},
    )
    print("Model IBM Granite berhasil diinisialisasi.")

REPLICATE_API_TOKEN loaded from Google Colab secret.

REPLICATE_API_TOKEN berhasil dimuat.
Model IBM Granite berhasil diinisialisasi.


In [ ]:
# --- Page App.py: Menggunakan Model untuk Generate Teks (Code Optimization) ---

if REPLICATE_API_TOKEN: # Pastikan token sudah dimuat sebelum mencoba menggunakan model
    print("\n--- Memulai Generasi Kode Aplikasi dengan IBM Granite ---")
    my_prompt = """
    Review the following Python Streamlit function. Suggest any optimizations for readability, performance, or best practices. If there are no major issues, just confirm it's good. Next, for me, you must generate the full-code version based on your suggestions.

    import streamlit as st
import json
import pandas as pd
from datetime import datetime

# Import dari file konfigurasi
from config import GOOGLE_API_KEY, get_gemini_model

# Import fungsi-fungsi utilitas
from utils.pdf_utils import generate_pdf_from_text, generate_analysis_pdf
from utils.gemini_utils import generate_narrative, generate_analysis_data
from utils.sidebar_content import render_custom_sidebar_content, render_sidebar_expander_content

# --- Konfigurasi API dan Model ---
try:
    gemini_model = get_gemini_model()
except Exception as e:
    st.error(f"Maaf, kami mengalami masalah teknis. Silakan coba lagi nanti atau hubungi pengembang.")
    st.stop()

# --- Streamlit UI Setup (Hanya di app.py) ---
st.set_page_config(
    page_title="Beranda Utama",
    layout="wide",
    initial_sidebar_state="expanded"
)

# --- Load Custom CSS (Hanya di app.py yang pertama kali memuatnya, tapi akan dimuat ulang di setiap halaman `pages/` juga) ---
def load_css(file_name):
    with open(file_name) as f:
        st.markdown(f'<style>{f.read()}</style>', unsafe_allow_html=True)

load_css('assets/style.css')

# --- Inisialisasi Session State di Awal Skrip ---
# Ini sangat penting agar variabel tidak hilang saat rerun
if 'generated_narration' not in st.session_state:
    st.session_state.generated_narration = ""
if 'generated_analysis' not in st.session_state:
    st.session_state.generated_analysis = {} # Gunakan dictionary kosong jika hasilnya JSON
if 'narasi_pdf_bytes' not in st.session_state:
    st.session_state.narasi_pdf_bytes = None
if 'analisis_pdf_bytes' not in st.session_state:
    st.session_state.analisis_pdf_bytes = None
if 'narasi_file_name' not in st.session_state:
    st.session_state.narasi_file_name = ""
if 'analisis_file_name' not in st.session_state:
    st.session_state.analisis_file_name = ""


# --- Sidebar ---
with st.sidebar:
    st.header("Nusantara Story")
    render_custom_sidebar_content()
    render_sidebar_expander_content()

# --- Main Content for app.py (Homepage) ---
st.title("Nusantara Story: Menggali Kisah Budaya, Memicu Potensi Wisata 🗺️")
st.markdown("Jelajahi potensi tak terbatas budaya dan pariwisata lokal Anda. Aplikasi ini dirancang untuk membantu Anda merangkai **narasi yang memikat** dan **strategi promosi cerdas**, didukung oleh kecerdasan buatan **Gemini-2.5 Flash** dan **IBM Granite**.")
st.markdown("---")

# Menggunakan st.form untuk input agar tidak langsung rerun saat input berubah
with st.form("story_generation_form"):
    col_input1, col_input2 = st.columns(2)

    with col_input1:
        st.markdown('<p style="font-weight: 600; color: #555555; margin-bottom: 5px;">Nama Objek Budaya/Wisata <span style="color:red">*</span></p>', unsafe_allow_html=True)
        judul_objek = st.text_input("", placeholder="Contoh: Kopi Gayo, Tari Saman, Candi Prambanan", key="input_judul", label_visibility="collapsed")
        st.markdown('<p class="custom-help-text">Nama spesifik objek yang ingin Anda ceritakan atau promosikan.</p>', unsafe_allow_html=True)

        st.markdown('<p style="font-weight: 600; color: #555555; margin-bottom: 5px;">Lokasi Obyek (Kota/Kabupaten/Provinsi) <span style="color:red">*</span></p>', unsafe_allow_html=True)
        lokasi_objek = st.text_input("", placeholder="Contoh: Aceh Tengah, Sumatra Utara, Bondowoso", key="input_lokasi", label_visibility="collapsed")
        st.markdown('<p class="custom-help-text">Lokasi geografis di mana objek ini berada.</p>', unsafe_allow_html=True)


    with col_input2:
        st.markdown('<p style="font-weight: 600; color: #555555; margin-bottom: 5px;">Pilih Gaya Bahasa Narasi (Opsional)</p>', unsafe_allow_html=True)
        gaya_bahasa = st.selectbox("", ["Pilih Gaya", "Edukasi", "Promosi", "Cerita Rakyat", "Puitis", "Informatif", "Inspiratif"], key="select_gaya", label_visibility="collapsed")
        st.markdown('<p class="custom-help-text">Pilih nuansa dan gaya penulisan yang Anda inginkan untuk narasi.</p>', unsafe_allow_html=True)

        st.markdown('<p style="font-weight: 600; color: #555555; margin-bottom: 5px;">Target Audiens Utama (Opsional)</p>', unsafe_allow_html=True)
        target_audiens = st.text_input("", value="", placeholder="Contoh: Wisatawan Keluarga, Pecinta Sejarah, Penggemar Kopi", key="input_target", label_visibility="collapsed")
        st.markdown('<p class="custom-help-text">Siapa target utama pesan promosi ini? (Misal: anak muda, keluarga, turis asing).</p>', unsafe_allow_html=True)

    st.markdown('<p style="font-weight: 600; color: #555555; margin-bottom: 5px;">Deskripsi Singkat / Poin-poin Kunci / Fakta Sejarah <span style="color:red">*</span></p>', unsafe_allow_html=True)
    deskripsi_kunci = st.text_area("", height=150,
                                    placeholder="Sebutkan detail penting, fragmen cerita, lokasi, tradisi, keunikan, atau fakta sejarah obyek ini. Semakin detail dan spesifik, semakin baik hasil yang akan AI berikan!",
                                    key="input_deskripsi", label_visibility="collapsed")
    st.markdown('<p class="custom-help-text">Ini adalah informasi inti untuk Kami merangkai cerita. Beri detail sebanyak mungkin!</p>', unsafe_allow_html=True)


    # --- Tombol Generate di dalam form ---
    submit_button = st.form_submit_button("Mulai Rangkai Kisah & Optimalkan Promosi! ✨", type="primary")

# --- Logika Setelah Tombol Submit Ditekan (di luar form agar bisa mengakses st.session_state) ---
if submit_button:
    if not judul_objek or not deskripsi_kunci or not lokasi_objek:
        st.warning("Mohon lengkapi semua kolom yang bertanda '*' (Wajib diisi) sebelum melanjutkan! 🙏")
        # Kosongkan session state jika input tidak valid agar output sebelumnya tidak muncul
        st.session_state.generated_narration = ""
        st.session_state.generated_analysis = {}
        st.session_state.narasi_pdf_bytes = None
        st.session_state.analisis_pdf_bytes = None
        st.session_state.narasi_file_name = ""
        st.session_state.analisis_file_name = ""
    else:
        # Hapus hasil sebelumnya dari session state untuk memastikan hasil baru
        st.session_state.generated_narration = ""
        st.session_state.generated_analysis = {}
        st.session_state.narasi_pdf_bytes = None
        st.session_state.analisis_pdf_bytes = None
        st.session_state.narasi_file_name = ""
        st.session_state.analisis_file_name = ""

        # --- Tahap 1: Generasi Narasi oleh Gemini ---
        # HILANGKAN st.subheader dan narasi_placeholder DI SINI
        # st.subheader("📝 Kisah & Narasi")
        # narasi_placeholder = st.empty()

        with st.spinner("Kami sedang menyusun narasi memukau untuk Anda... Sabar ya! ⏳"):
            generated_narration = generate_narrative(
                gemini_model, judul_objek, lokasi_objek, deskripsi_kunci, target_audiens, gaya_bahasa
            )

            if generated_narration:
                st.session_state.generated_narration = generated_narration # Simpan ke session state

                # Generate PDF bytes dan simpan juga ke session state
                pdf_bytes_narasi_temp = generate_pdf_from_text(generated_narration, f"Narasi_{judul_objek}")
                if pdf_bytes_narasi_temp:
                    st.session_state.narasi_pdf_bytes = pdf_bytes_narasi_temp
                    st.session_state.narasi_file_name = f"Kisah_{judul_objek}.pdf"
                else:
                    st.error("Gagal membuat PDF Narasi.")
                    st.session_state.narasi_pdf_bytes = None
                    st.session_state.narasi_file_name = ""

            else:
                # narasi_placeholder.error("Maaf, Kami gagal merangkai narasi yang valid. Coba ulangi atau sesuaikan input Anda.")
                st.session_state.generated_narration = "" # Kosongkan jika gagal
                st.error("Maaf, Kami gagal merangkai narasi yang valid. Coba ulangi atau sesuaikan input Anda.") # Tampilkan error di bagian bawah

        # --- Tahap 2: Analisis & Optimasi oleh Gemini ---
        if st.session_state.generated_narration:
            with st.spinner("Kami sedang menganalisis potensi tak terbatas destinasi Anda... Mohon tunggu! 🚀"):
                analysis_data = generate_analysis_data(gemini_model, lokasi_objek, st.session_state.generated_narration)

                if analysis_data:
                    st.session_state.generated_analysis = analysis_data # Simpan ke session state

                    # Generate PDF bytes untuk analisis dan simpan juga ke session state
                    pdf_bytes_analysis_temp = generate_analysis_pdf(analysis_data, f"Analisis_{judul_objek}")
                    if pdf_bytes_analysis_temp:
                        st.session_state.analisis_pdf_bytes = pdf_bytes_analysis_temp
                        st.session_state.analisis_file_name = f"Analisis_Promosi_{judul_objek}.pdf"
                    else:
                        st.error("Gagal membuat PDF Analisis Promosi.")
                        st.session_state.analisis_pdf_bytes = None
                        st.session_state.analisis_file_name = ""

                else:
                    # st.error("Maaf, Kami gagal mendapatkan analisis yang valid. Coba ulangi atau sesuaikan input Anda.")
                    st.session_state.generated_analysis = {} # Kosongkan jika gagal
                    st.error("Maaf, Kami gagal mendapatkan analisis yang valid. Coba ulangi atau sesuaikan input Anda.") # Tampilkan error di bagian bawah
        else:
            st.warning("Analisis tidak dapat dilakukan karena narasi belum berhasil dibuat.")

# --- Tampilkan Hasil dan Tombol Unduh (di luar blok `if submit_button`) ---
# Bagian ini adalah SATU-SATUNYA tempat hasil dan tombol download akan muncul

# Tampilkan narasi hanya jika ada di session state
if st.session_state.generated_narration:
    st.subheader("📝 Kisah & Narasi")
    st.markdown(f"<div class='output-card'><p>{st.session_state.generated_narration}</p></div>", unsafe_allow_html=True)

    # Tombol Unduh Narasi PDF
    if st.session_state.narasi_pdf_bytes:
        st.download_button(
            label="Unduh Naskah Cerita (PDF) ⬇️",
            data=st.session_state.narasi_pdf_bytes,
            file_name=st.session_state.narasi_file_name,
            mime="application/pdf",
            key="download_narasi_pdf_final", # Key unik
            help="Unduh naskah cerita sebagai file PDF. Siap untuk dibagikan!"
        )

# Tampilkan analisis hanya jika ada di session state
if st.session_state.generated_analysis:
    st.markdown("---")
    st.subheader("💡 Wawasan & Optimasi Promosi Wisata")

    # Render ulang analisis di kolom
    col_analysis1_rerun, col_analysis2_rerun = st.columns(2)
    col1_keys_rerun = [
        "Poin Jual Utama",
        "Segmen Wisatawan Ideal",
        "Ide Monetisasi & Produk Pariwisata"
    ]
    col2_keys_rerun = [
        "Saran Peningkatan Pesan Promosi",
        "Potensi Kolaborasi Lokal"
    ]

    with col_analysis1_rerun:
        for key in col1_keys_rerun:
            if key in st.session_state.generated_analysis and st.session_state.generated_analysis[key]:
                st.markdown(f'<div class="info-card">', unsafe_allow_html=True)
                st.markdown(f"<h4>{key}</h4>", unsafe_allow_html=True)
                for item in st.session_state.generated_analysis[key]:
                    if 'poin' in item:
                        st.markdown(f"**👉 {item['poin']}**", unsafe_allow_html=True)
                    if 'deskripsi' in item:
                        st.write(item['deskripsi'])
                st.markdown('</div>', unsafe_allow_html=True)

    with col_analysis2_rerun:
        for key in col2_keys_rerun:
            if key in st.session_state.generated_analysis and st.session_state.generated_analysis[key]:
                st.markdown(f'<div class="info-card">', unsafe_allow_html=True)
                st.markdown(f"<h4>{key}</h4>", unsafe_allow_html=True)
                for item in st.session_state.generated_analysis[key]:
                    if 'poin' in item:
                        st.markdown(f"**👉 {item['poin']}**", unsafe_allow_html=True)
                    if 'deskripsi' in item:
                        st.write(item['deskripsi'])
                st.markdown('</div>', unsafe_allow_html=True)

    # Tombol Unduh Analisis Promosi PDF
    if st.session_state.analisis_pdf_bytes:
        st.download_button(
            label="Unduh Analisis Promosi (PDF) ⬇️",
            data=st.session_state.analisis_pdf_bytes,
            file_name=st.session_state.analisis_file_name,
            mime="application/pdf",
            key="download_analysis_pdf_final", # Key unik
            help="Dapatkan dokumen analisis lengkap untuk panduan promosi Anda!"
        )


# --- Footer Copyright ---
st.markdown("---")
st.markdown(f"<p style='text-align: center; color: #777;'>© {datetime.now().year} Nusantara Story. Dibuat dengan ✨ oleh Kholish Fauzan.</p>", unsafe_allow_html=True)
    """

    print(f"\nMemproses permintaan Anda dengan IBM Granite untuk menghasilkan kode aplikasi...")

    try:
        # Panggil model IBM Granite dengan prompt yang sudah didefinisikan dalam variabel
        generated_code_output = model.invoke(my_prompt)

        print("\n--- Hasil Generasi Kode dari IBM Granite ---")
        print(generated_code_output)
        print("\n-------------------------------------------")

    except Exception as e:
        print(f"Terjadi kesalahan saat memanggil model IBM Granite: {e}")
        print("Pastikan prompt tidak terlalu panjang atau ada masalah dengan API token Replicate Anda.")

else:
    print("\nModel IBM Granite tidak dapat digunakan karena REPLICATE_API_TOKEN tidak valid atau tidak dimuat.")


--- Memulai Generasi Kode Aplikasi dengan IBM Granite ---

Memproses permintaan Anda dengan IBM Granite untuk menghasilkan kode aplikasi...

--- Hasil Generasi Kode dari IBM Granite ---
The provided Streamlit code appears to be well-structured and follows best practices for readability and organization. Here are a few minor suggestions for optimization:

1. **Error Handling**: The code already includes error handling for API and model initialization. Consider adding more specific exception types for better debugging.

2. **Code Duplication**: There's some code duplication in the form of similar markdown formatting for help texts. Consider creating a function to generate these markdown snippets to avoid repetition.

3. **Session State Management**: The code already initializes session states for various variables. Make sure these states are cleared or updated appropriately in case of errors or reruns.

4. **PDF Generation**: The code generates PDFs for both narration and analysis. Cons

In [ ]:
# --- Page App.py: Menggunakan Model untuk Generate Teks (Code Optimization - Checking Error) ---

if REPLICATE_API_TOKEN: # Pastikan token sudah dimuat sebelum mencoba menggunakan model
    print("\n--- Memulai Generasi Kode Aplikasi dengan IBM Granite ---")
    my_prompt = """
    By this code, I got a new error which is:
    "NameError: This app has encountered an error. The original error message is redacted to prevent data leaks. Full error details have been recorded in the logs (if you're on Streamlit Cloud, click on 'Manage app' in the lower right of your app).
Traceback:
File "/mount/src/hacktiv8-project/app.py", line 133, in <module>
    st.session_state.generated_analysis = analysis_data # Simpan ke session state
                                          ^^^^^^^^^^^^^"

    As I know that this analysis_data is not defined priorly in your code. I got this previously from your code suggestion. Suggest any optimizations for fixes, readability, performance, or best practices. If there are no major issues, just confirm it's good. Next, for me, you must generate the full-code version based on your suggestions.

import streamlit as st
import json
import pandas as pd
from datetime import datetime

# Import dari file konfigurasi
from config import GOOGLE_API_KEY, get_gemini_model

# Import fungsi-fungsi utilitas
from utils.pdf_utils import generate_pdf_from_text, generate_analysis_pdf
from utils.gemini_utils import generate_narrative, generate_analysis_data
from utils.sidebar_content import render_custom_sidebar_content, render_sidebar_expander_content

# --- Konfigurasi API dan Model ---
try:
    gemini_model = get_gemini_model()
except Exception as e:
    st.error(f"Maaf, kami mengalami masalah teknis. Silakan coba lagi nanti atau hubungi pengembang.")
    st.stop()

# --- Streamlit UI Setup (Hanya di app.py) ---
st.set_page_config(
    page_title="Beranda Utama",
    layout="wide",
    initial_sidebar_state="expanded"
)

# --- Load Custom CSS (Hanya di app.py yang pertama kali memuatnya, tapi akan dimuat ulang di setiap halaman `pages/` juga) ---
def load_css(file_name):
    with open(file_name) as f:
        st.markdown(f'<style>{f.read()}</style>', unsafe_allow_html=True)

load_css('assets/style.css')

# --- Inisialisasi Session State di Awal Skrip ---
if 'generated_narration' not in st.session_state:
    st.session_state.generated_narration = ""
if 'generated_analysis' not in st.session_state:
    st.session_state.generated_analysis = {}
if 'narasi_pdf_bytes' not in st.session_state:
    st.session_state.narasi_pdf_bytes = None
if 'analisis_pdf_bytes' not in st.session_state:
    st.session_state.analisis_pdf_bytes = None
if 'narasi_file_name' not in st.session_state:
    st.session_state.narasi_file_name = ""
if 'analisis_file_name' not in st.session_state:
    st.session_state.analisis_file_name = ""


# --- Sidebar ---
with st.sidebar:
    st.header("Nusantara Story")
    render_custom_sidebar_content()
    render_sidebar_expander_content()

# --- Main Content for app.py (Homepage) ---
st.title("Nusantara Story: Menggali Kisah Budaya, Memicu Potensi Wisata 🗺️")
st.markdown("Jelajahi potensi tak terbatas budaya dan pariwisata lokal Anda. Aplikasi ini dirancang untuk membantu Anda merangkai **narasi yang memikat** dan **strategi promosi cerdas**, didukung oleh kecerdasan buatan **Gemini-2.5 Flash** dan **IBM Granite**.")

# Menggunakan st.form untuk input agar tidak langsung rerun saat input berubah
with st.form("story_generation_form"):
    col_input1, col_input2 = st.columns(2)

    with col_input1:
        st.markdown('<p style="font-weight: 600; color: #555555; margin-bottom: 5px;">Nama Objek Budaya/Wisata <span style="color:red">*</span></p>', unsafe_allow_html=True)
        judul_objek = st.text_input("", placeholder="Contoh: Kopi Gayo, Tari Saman, Candi Prambanan", key="input_judul", label_visibility="collapsed")
        st.markdown('<p class="custom-help-text">Nama spesifik objek yang ingin Anda ceritakan atau promosikan.</p>', unsafe_allow_html=True)

        st.markdown('<p style="font-weight: 600; color: #555555; margin-bottom: 5px;">Lokasi Obyek (Kota/Kabupaten/Provinsi) <span style="color:red">*</span></p>', unsafe_allow_html=True)
        lokasi_objek = st.text_input("", placeholder="Contoh: Aceh Tengah, Sumatra Utara, Bondowoso", key="input_lokasi", label_visibility="collapsed")
        st.markdown('<p class="custom-help-text">Lokasi geografis di mana objek ini berada.</p>', unsafe_allow_html=True)


    with col_input2:
        st.markdown('<p style="font-weight: 600; color: #555555; margin-bottom: 5px;">Pilih Gaya Bahasa Narasi (Opsional)</p>', unsafe_allow_html=True)
        gaya_bahasa = st.selectbox("", ["Pilih Gaya", "Edukasi", "Promosi", "Cerita Rakyat", "Puitis", "Informatif", "Inspiratif"], key="select_gaya", label_visibility="collapsed")
        st.markdown('<p class="custom-help-text">Pilih nuansa dan gaya penulisan yang Anda inginkan untuk narasi.</p>', unsafe_allow_html=True)

        st.markdown('<p style="font-weight: 600; color: #555555; margin-bottom: 5px;">Target Audiens Utama (Opsional)</p>', unsafe_allow_html=True)
        target_audiens = st.text_input("", value="", placeholder="Contoh: Wisatawan Keluarga, Pecinta Sejarah, Penggemar Kopi", key="input_target", label_visibility="collapsed")
        st.markdown('<p class="custom-help-text">Siapa target utama pesan promosi ini? (Misal: anak muda, keluarga, turis asing).</p>', unsafe_allow_html=True)

    st.markdown('<p style="font-weight: 600; color: #555555; margin-bottom: 5px;">Deskripsi Singkat / Poin-poin Kunci / Fakta Sejarah <span style="color:red">*</span></p>', unsafe_allow_html=True)
    deskripsi_kunci = st.text_area("", height=150,
                                    placeholder="Sebutkan detail penting, fragmen cerita, lokasi, tradisi, keunikan, atau fakta sejarah obyek ini. Semakin detail dan spesifik, semakin baik hasil yang akan AI berikan!",
                                    key="input_deskripsi", label_visibility="collapsed")
    st.markdown('<p class="custom-help-text">Ini adalah informasi inti untuk Kami merangkai cerita. Beri detail sebanyak mungkin!</p>', unsafe_allow_html=True)


    # --- Tombol Generate di dalam form ---
    submit_button = st.form_submit_button("Mulai Rangkai Kisah & Optimalkan Promosi! ✨", type="primary")

# --- Logika Setelah Tombol Submit Ditekan (di luar form agar bisa mengakses st.session_state) ---
if submit_button:
    if not judul_objek or not deskripsi_kunci or not lokasi_objek:
        st.warning("Mohon lengkapi semua kolom yang bertanda '*' (Wajib diisi) sebelum melanjutkan! 🙏")
    else:
        # Hapus hasil sebelumnya dari session state untuk memastikan hasil baru
        st.session_state.generated_narration = ""
        st.session_state.generated_analysis = {}
        st.session_state.narasi_pdf_bytes = None
        st.session_state.analisis_pdf_bytes = None
        st.session_state.narasi_file_name = ""
        st.session_state.analisis_file_name = ""

        # --- Tahap 1: Generasi Narasi oleh Gemini ---
        with st.spinner("Kami sedang menyusun narasi memukau untuk Anda... Sabar ya! ⏳"):
            generated_narration = generate_narrative(
                gemini_model, judul_objek, lokasi_objek, deskripsi_kunci, target_audiens, gaya_bahasa
            )

            if generate_narrative(gemini_model, judul_objek, lokasi_objek, deskripsi_kunci, target_audiens, gaya_bahasa):
                st.session_state.generated_narration = generated_narration

                # Generate PDF bytes dan simpan juga ke session state
                pdf_bytes_narasi_temp = generate_pdf_from_text(generated_narration, f"Narasi_{judul_objek}")
                if pdf_bytes_narasi_temp:
                    st.session_state.narasi_pdf_bytes = pdf_bytes_narasi_temp
                    st.session_state.narasi_file_name = f"Kisah_{judul_objek}.pdf"
                else:
                    st.error("Gagal membuat PDF Narasi.")
                    st.session_state.narasi_pdf_bytes = None
                    st.session_state.narasi_file_name = ""

            else:
                # narasi_placeholder.error("Maaf, Kami gagal merangkai narasi yang valid. Coba ulangi atau sesuaikan input Anda.")
                st.session_state.generated_narration = "" # Kosongkan jika gagal
                st.error("Maaf, Kami gagal merangkai narasi yang valid. Coba ulangi atau sesuaikan input Anda.") # Tampilkan error di bagian bawah

        # --- Tahap 2: Analisis & Optimasi oleh Gemini ---
        if st.session_state.generated_narration:
            if generate_analysis_data(gemini_model, lokasi_objek, st.session_state.generated_narration):
                    st.session_state.generated_analysis = analysis_data # Simpan ke session state

                    # Generate PDF bytes untuk analisis dan simpan juga ke session state
                    pdf_bytes_analysis_temp = generate_analysis_pdf(analysis_data, f"Analisis_{judul_objek}")
                    if pdf_bytes_analysis_temp:
                        st.session_state.analisis_pdf_bytes = pdf_bytes_analysis_temp
                        st.session_state.analisis_file_name = f"Analisis_Promosi_{judul_objek}.pdf"
                    else:
                        st.error("Gagal membuat PDF Analisis Promosi.")
                        st.session_state.analisis_pdf_bytes = None
                        st.session_state.analisis_file_name = ""
            else:
                # st.error("Maaf, Kami gagal mendapatkan analisis yang valid. Coba ulangi atau sesuaikan input Anda.")
                st.session_state.generated_analysis = {} # Kosongkan jika gagal
                st.error("Maaf, Kami gagal mendapatkan analisis yang valid. Coba ulangi atau sesuaikan input Anda.") # Tampilkan error di bagian bawah
        else:
            st.warning("Analisis tidak dapat dilakukan karena narasi belum berhasil dibuat.")

# --- Tampilkan Hasil dan Tombol Unduh (di luar blok `if submit_button`) ---

# Tampilkan narasi hanya jika ada di session state
if st.session_state.generated_narration:
    st.subheader("📝 Kisah & Narasi")
    st.markdown(f"<div class='output-card'><p>{st.session_state.generated_narration}</p></div>", unsafe_allow_html=True)

    # Tombol Unduh Narasi PDF
    if st.session_state.narasi_pdf_bytes:
        st.download_button(
            label="Unduh Naskah Cerita (PDF) ⬇️",
            data=st.session_state.narasi_pdf_bytes,
            file_name=st.session_state.narasi_file_name,
            mime="application/pdf",
            key="download_narasi_pdf_final", # Key unik
            help="Unduh naskah cerita sebagai file PDF. Siap untuk dibagikan!"
        )

# Tampilkan analisis hanya jika ada di session state
if st.session_state.generated_analysis:
    st.markdown("---")
    st.subheader("💡 Wawasan & Optimasi Promosi Wisata")

    # Render ulang analisis di kolom
    col_analysis1_rerun, col_analysis2_rerun = st.columns(2)
    col1_keys_rerun = [
        "Poin Jual Utama",
        "Segmen Wisatawan Ideal",
        "Ide Monetisasi & Produk Pariwisata"
    ]
    col2_keys_rerun = [
        "Saran Peningkatan Pesan Promosi",
        "Potensi Kolaborasi Lokal"
    ]

    with col_analysis1_rerun:
        for key in col1_keys_rerun:
            if key in st.session_state.generated_analysis and st.session_state.generated_analysis[key]:
                st.markdown(f'<div class="info-card">', unsafe_allow_html=True)
                st.markdown(f"<h4>{key}</h4>", unsafe_allow_html=True)
                for item in st.session_state.generated_analysis[key]:
                    if 'poin' in item:
                        st.markdown(f"**👉 {item['poin']}**", unsafe_allow_html=True)
                    if 'deskripsi' in item:
                        st.write(item['deskripsi'])
                st.markdown('</div>', unsafe_allow_html=True)

    with col_analysis2_rerun:
        for key in col2_keys_rerun:
            if key in st.session_state.generated_analysis and st.session_state.generated_analysis[key]:
                st.markdown(f'<div class="info-card">', unsafe_allow_html=True)
                st.markdown(f"<h4>{key}</h4>", unsafe_allow_html=True)
                for item in st.session_state.generated_analysis[key]:
                    if 'poin' in item:
                        st.markdown(f"**👉 {item['poin']}**", unsafe_allow_html=True)
                    if 'deskripsi' in item:
                        st.write(item['deskripsi'])
                st.markdown('</div>', unsafe_allow_html=True)

    # Tombol Unduh Analisis Promosi PDF
    if st.session_state.analisis_pdf_bytes:
        st.download_button(
            label="Unduh Analisis Promosi (PDF) ⬇️",
            data=st.session_state.analisis_pdf_bytes,
            file_name=st.session_state.analisis_file_name,
            mime="application/pdf",
            key="download_analysis_pdf_final", # Key unik
            help="Dapatkan dokumen analisis lengkap untuk panduan promosi Anda!"
        )


# --- Footer Copyright ---
st.markdown("---")
st.markdown(f"<p style='text-align: center; color: #777;'>© {datetime.now().year} Nusantara Story. Dibuat dengan ✨ oleh Kholish Fauzan.</p>", unsafe_allow_html=True)
    """

    print(f"\nMemproses permintaan Anda dengan IBM Granite untuk menghasilkan kode aplikasi...")

    try:
        # Panggil model IBM Granite dengan prompt yang sudah didefinisikan dalam variabel
        generated_code_output = model.invoke(my_prompt)

        print("\n--- Hasil Generasi Kode dari IBM Granite ---")
        print(generated_code_output)
        print("\n-------------------------------------------")

    except Exception as e:
        print(f"Terjadi kesalahan saat memanggil model IBM Granite: {e}")
        print("Pastikan prompt tidak terlalu panjang atau ada masalah dengan API token Replicate Anda.")

else:
    print("\nModel IBM Granite tidak dapat digunakan karena REPLICATE_API_TOKEN tidak valid atau tidak dimuat.")


--- Memulai Generasi Kode Aplikasi dengan IBM Granite ---

Memproses permintaan Anda dengan IBM Granite untuk menghasilkan kode aplikasi...

--- Hasil Generasi Kode dari IBM Granite ---
The error you're encountering is a `NameError` because `analysis_data` is not defined before it's assigned to `st.session_state.generated_analysis`. To fix this issue, ensure that `analysis_data` is generated and defined before this line.

Looking at your code, the function `generate_analysis_data(gemini_model, lokasi_objek, st.session_state.generated_narration)` is called, but the return value (i.e., `analysis_data`) is not captured or assigned to a variable. To resolve this, you should capture the return value of `generate_analysis_data()` and assign it to `analysis_data`.

Here's the updated code snippet with the fix:

```python
# --- Tahap 2: Analisis & Optimasi oleh Gemini ---
if st.session_state.generated_narration: 
    analysis_data = generate_analysis_data(gemini_model, lokasi_objek, st.sessio

In [ ]:
# --- Page Config.py: Menggunakan Model untuk Generate Teks (Code Optimization) ---

if REPLICATE_API_TOKEN: # Pastikan token sudah dimuat sebelum mencoba menggunakan model
    print("\n--- Memulai Generasi Kode Aplikasi dengan IBM Granite ---")
    my_prompt = """
    Review the following Python Streamlit function. Suggest any optimizations for readability, performance, or best practices. If there are no major issues, just confirm it's good. Next, for me, you must generate the full-code version based on your suggestions.
    You don't have to worry about my GOOGLE_API_KEY because it's already stored in Streamlit Cloud Secrets. Just focus on refining the code if possible.
    If it's possible, is there a code so that the text generation is also faster? by using batching or caching instead?
    Or maybe optimize the result by using parameters like temperature, top_p, etc?

# config.py
import streamlit as st
import google.generativeai as genai
import os

# Google Gemini API Key
# Mengambil dari Streamlit Cloud Secrets atau environment variable lokal
try:
    GOOGLE_API_KEY = st.secrets["GOOGLE_API_KEY"]
except KeyError:
    st.error("Google Gemini API key tidak ditemukan di Streamlit Secrets. Pastikan sudah diatur.")
    st.stop()

# Fungsi untuk mendapatkan model Gemini yang dikonfigurasi
def get_gemini_model():
    genai.configure(api_key=GOOGLE_API_KEY)
    return genai.GenerativeModel('gemini-2.5-flash')
    """

    print(f"\nMemproses permintaan Anda dengan IBM Granite untuk menghasilkan kode aplikasi...")

    try:
        # Panggil model IBM Granite dengan prompt yang sudah didefinisikan dalam variabel
        generated_code_output = model.invoke(my_prompt)

        print("\n--- Hasil Generasi Kode dari IBM Granite ---")
        print(generated_code_output)
        print("\n-------------------------------------------")

    except Exception as e:
        print(f"Terjadi kesalahan saat memanggil model IBM Granite: {e}")
        print("Pastikan prompt tidak terlalu panjang atau ada masalah dengan API token Replicate Anda.")

else:
    print("\nModel IBM Granite tidak dapat digunakan karena REPLICATE_API_TOKEN tidak valid atau tidak dimuat.")


--- Memulai Generasi Kode Aplikasi dengan IBM Granite ---

Memproses permintaan Anda dengan IBM Granite untuk menghasilkan kode aplikasi...

--- Hasil Generasi Kode dari IBM Granite ---
The provided code is already well-structured and follows good practices. It securely retrieves the API key from Streamlit Cloud Secrets and configures the Google Gemini model. Here are a few minor suggestions for optimization and readability:

1. Add docstrings to functions for better documentation.
2. Use a context manager for handling exceptions to clean up resources if needed.
3. For text generation, you can use the `genai.GenerativeModel.generate()` method with parameters like `temperature` and `top_p` to control the output. However, batching or caching would depend on the specific use case and might not be applicable for a single text generation function.

Here's the optimized code with the suggested changes:

```python
# config.py
import streamlit as st
import google.generativeai as genai
import 

In [ ]:
# --- Code "gemini_utils.py": Menggunakan Model untuk Generate Teks (Code Optimization) ---

if REPLICATE_API_TOKEN: # Pastikan token sudah dimuat sebelum mencoba menggunakan model
    print("\n--- Memulai Generasi Kode Aplikasi dengan IBM Granite ---")
    my_prompt = """
Review the following Python Streamlit function. Suggest any optimizations for readability, performance, or best practices.
If there are no major issues, just confirm it's good. Next, for me, you must generate the full-code version based on your suggestions.

Here is the code (in Markdown code block):

```python
import json
import streamlit as st  # Streamlit di sini tidak digunakan untuk UI, tapi untuk st.error

# Tambahkan import untuk google.generativeai
import google.generativeai as genai

# Fungsi pembantu untuk mendapatkan nama bahasa dari target audiens
def get_language_from_audience(target_audiens):
    target_audiens_lower = target_audiens.lower()
    if "inggris" in target_audiens_lower or "english" in target_audiens_lower or "foreign" in target_audiens_lower or "asing" in target_audiens_lower:
        return "English"
    elif "china" in target_audiens_lower or "mandarin" in target_audiens_lower:
        return "Chinese (Simplified)"
    elif "jepang" in target_audiens_lower:
        return "Japanese"
    elif "korea" in target_audiens_lower:
        return "Korean"
    elif "jerman" in target_audiens_lower:
        return "German"
    elif "perancis" in target_audiens_lower or "prancis" in target_audiens_lower:
        return "French"
    elif "spanyol" in target_audiens_lower:
        return "Spanish"
    elif "arab" in target_audiens_lower:
        return "Arabic"
    else:
        return "Indonesian"

def generate_narrative(model, judul_objek, lokasi_objek, deskripsi_kunci, target_audiens, gaya_bahasa):
    target_language = get_language_from_audience(target_audiens)
    gaya_prompt = f"Gaya Bahasa (jika dipilih): {gaya_bahasa if gaya_bahasa != 'Pilih Gaya' else 'Informatif dan Menarik'}"

    prompt = f\"""
    Anda adalah seorang ahli narasi budaya dan pariwisata Indonesia. Buatlah narasi yang memukau dan informatif tentang objek budaya/pariwisata berikut.
    **Instruksi Penting**: Hasilkan output sepenuhnya dalam bahasa {target_language}.

    Nama Objek: {judul_objek}
    Lokasi: {lokasi_objek}
    Deskripsi Kunci/Fakta Sejarah: {deskripsi_kunci}

    Target Audiens (jika ada): {target_audiens if target_audiens else 'Umum'}
    {gaya_prompt}

    Fokuskan pada:
    1. Keunikan dan daya tarik utama objek tersebut.
    2. Sejarah atau latar belakang budaya yang relevan (jika ada dalam deskripsi kunci).
    3. Potensi pengalaman bagi pengunjung/pembaca.
    4. Gunakan bahasa yang kaya dan deskriptif.
    5. Panjang narasi sekitar 400-600 kata.

    Pastikan narasi tersebut otentik dan menggugah minat.
    \"""

    try:
        response = model.generate_content(prompt)
        return response.text
    except Exception as e:
        st.error(f"Terjadi kesalahan saat menghasilkan narasi: {e}. Pastikan API Key valid dan model berfungsi.")
        return None

def generate_analysis_data(model, lokasi_objek, narrative_text):
    prompt = f\"""
    Berisi analisis mendalam yang berfokus pada potensi promosi dan pengembangan ekonomi lokal berdasarkan narasi tentang objek budaya/pariwisata di {lokasi_objek} ini.
    Berikan output dalam format JSON yang terstruktur dengan kunci-kunci berikut. Untuk setiap kunci, berikan minimal 3 poin (jika relevan).

    Narasi:
    {narrative_text}

    ```json
    {{
      "Poin Jual Utama": [
        {{"poin": "Poin utama 1", "deskripsi": "Deskripsi poin 1"}},
        {{"poin": "Poin utama 2", "deskripsi": "Deskripsi poin 2"}}
      ],
      "Segmen Wisatawan Ideal": [
        {{"poin": "Segmen 1", "deskripsi": "Deskripsi segmen 1"}},
        {{"poin": "Segmen 2", "deskripsi": "Deskripsi segmen 2"}}
      ],
      "Ide Monetisasi & Produk Pariwisata": [
        {{"poin": "Ide 1", "deskripsi": "Deskripsi ide 1"}},
        {{"poin": "Ide 2", "deskripsi": "Deskripsi ide 2"}}
      ],
      "Saran Peningkatan Pesan Promosi": [
        {{"poin": "Saran 1", "deskripsi": "Deskripsi saran 1"}},
        {{"poin": "Saran 2", "deskripsi": "Deskripsi saran 2"}}
      ],
      "Potensi Kolaborasi Lokal": [
        {{"poin": "Kolaborasi 1", "deskripsi": "Deskripsi kolaborasi 1"}},
        {{"poin": "Kolaborasi 2", "deskripsi": "Deskripsi kolaborasi 2"}}
      ]
    }}
    ```

    Pastikan output adalah JSON yang valid dan dapat di-parse langsung.
    \"""

    try:
        response = model.generate_content(prompt)
        json_text = response.text.strip()
        if json_text.startswith("```json"):
            json_text = json_text[len("```json"):].strip()
        if json_text.endswith("```"):
            json_text = json_text[:-len("```")].strip()

        return json.loads(json_text)
    except json.JSONDecodeError as e:
        st.error(f"Gagal mengurai respons AI sebagai JSON. Mohon coba lagi. Error: {e}")
        st.write("Respons AI mentah (untuk debugging):", response.text)
        return None
    except Exception as e:
        st.error(f"Terjadi kesalahan saat menghasilkan analisis: {e}. Pastikan API Key valid dan model berfungsi.")
        return None
"""

    print(f"\nMemproses permintaan Anda dengan IBM Granite untuk menghasilkan kode aplikasi...")

    try:
        # Panggil model IBM Granite dengan prompt yang sudah didefinisikan dalam variabel
        generated_code_output = model.invoke(my_prompt)

        print("\n--- Hasil Generasi Kode dari IBM Granite ---")
        print(generated_code_output)
        print("\n-------------------------------------------")

    except Exception as e:
        print(f"Terjadi kesalahan saat memanggil model IBM Granite: {e}")
        print("Pastikan prompt tidak terlalu panjang atau ada masalah dengan API token Replicate Anda.")

else:
    print("\nModel IBM Granite tidak dapat digunakan karena REPLICATE_API_TOKEN tidak valid atau tidak dimuat.")


--- Memulai Generasi Kode Aplikasi dengan IBM Granite ---

Memproses permintaan Anda dengan IBM Granite untuk menghasilkan kode aplikasi...

--- Hasil Generasi Kode dari IBM Granite ---
### Review and Suggestions

1. **Imports**: The import for `google.generativeai` seems unnecessary as there's no usage or reference to it in the provided functions. Remove this import unless it's being used elsewhere in the codebase.

2. **Function `get_language_from_audience`**: This function is straightforward but could benefit from a more efficient lookup method. Consider using a dictionary to map audience keywords to language codes. This will reduce redundancy and make future updates easier.

3. **String Formatting**: In Python 3.6 and above, f-strings are preferred for readability. Your use of f-strings is already good, but ensure consistency across similar string formatting.

4. **Error Handling**: The `try-except` blocks are comprehensive, but consider logging the exceptions for better debugging

In [ ]:
# --- Code "pdf_utils.py": Menggunakan Model untuk Generate Teks (Code Optimization) ---

if REPLICATE_API_TOKEN: # Pastikan token sudah dimuat sebelum mencoba menggunakan model
    print("\n--- Memulai Generasi Kode Aplikasi dengan IBM Granite ---")
    my_prompt = """
Review the following Python code that generates PDFs using ReportLab for a Streamlit application. Suggest any improvements regarding clarity, performance, modularity, or formatting best practices. If no improvements are needed, confirm it's solid. After that, please regenerate the full optimized code if any suggestion is applied.

```python
from reportlab.lib.pagesizes import letter
from reportlab.platypus import SimpleDocTemplate, Paragraph, Spacer, Frame, PageBreak
from reportlab.lib.styles import getSampleStyleSheet, ParagraphStyle
from reportlab.lib.enums import TA_CENTER, TA_LEFT
from reportlab.lib.units import inch
from io import BytesIO
from datetime import datetime  # Pastikan baris ini ada dan tidak dihapus!

def generate_pdf_from_text(text_content, title="Dokumen Streamlit"):
    buffer = BytesIO()
    doc = SimpleDocTemplate(buffer, pagesize=letter)
    styles = getSampleStyleSheet()
    story = []

    title_style = ParagraphStyle(
        'TitleStyle',
        parent=styles['h1'],
        fontSize=24,
        leading=28,
        alignment=TA_CENTER,
        spaceAfter=20
    )

    normal_style = ParagraphStyle(
        'NormalStyle',
        parent=styles['Normal'],
        fontSize=12,
        leading=14,
        alignment=TA_LEFT,
        spaceAfter=12
    )

    story.append(Paragraph(title, title_style))
    story.append(Spacer(1, 0.2 * inch))
    story.append(Paragraph(text_content.replace('\\n', '<br/>'), normal_style))

    try:
        doc.build(story)
        buffer.seek(0)
        return buffer.getvalue()
    except Exception as e:
        print(f"Error generating PDF: {e}")
        return None

def generate_analysis_pdf(analysis_data, title="Analisis Promosi"):
    buffer = BytesIO()
    doc = SimpleDocTemplate(buffer, pagesize=letter)
    styles = getSampleStyleSheet()
    story = []

    title_style = ParagraphStyle(
        'TitleStyle',
        parent=styles['h1'],
        fontSize=24,
        leading=28,
        alignment=TA_CENTER,
        spaceAfter=20,
        textColor='#1ABC9C'
    )
    section_title_style = ParagraphStyle(
        'SectionTitleStyle',
        parent=styles['h2'],
        fontSize=18,
        leading=22,
        spaceAfter=10,
        textColor='#34495E'
    )
    point_style = ParagraphStyle(
        'PointStyle',
        parent=styles['h3'],
        fontSize=14,
        leading=16,
        spaceBefore=10,
        spaceAfter=5,
        textColor='#2C3E50'
    )
    description_style = ParagraphStyle(
        'DescriptionStyle',
        parent=styles['Normal'],
        fontSize=11,
        leading=13,
        spaceAfter=10,
        leftIndent=20
    )
    footer_style = ParagraphStyle(
        'FooterStyle',
        parent=styles['Normal'],
        fontSize=9,
        alignment=TA_CENTER,
        textColor='#777777',
        spaceBefore=30
    )

    story.append(Paragraph(title, title_style))
    story.append(Spacer(1, 0.3 * inch))

    story.append(Paragraph("Poin Jual Utama", section_title_style))
    story.append(Spacer(1, 0.1 * inch))
    if "Poin Jual Utama" in analysis_data:
        for item in analysis_data["Poin Jual Utama"]:
            story.append(Paragraph(f"👉 {item.get('poin', '')}", point_style))
            story.append(Paragraph(item.get('deskripsi', ''), description_style))
    story.append(Spacer(1, 0.2 * inch))

    story.append(Paragraph("Segmen Wisatawan Ideal", section_title_style))
    story.append(Spacer(1, 0.1 * inch))
    if "Segmen Wisatawan Ideal" in analysis_data:
        for item in analysis_data["Segmen Wisatawan Ideal"]:
            story.append(Paragraph(f"👉 {item.get('poin', '')}", point_style))
            story.append(Paragraph(item.get('deskripsi', ''), description_style))
    story.append(Spacer(1, 0.2 * inch))

    story.append(Paragraph("Ide Monetisasi & Produk Pariwisata", section_title_style))
    story.append(Spacer(1, 0.1 * inch))
    if "Ide Monetisasi & Produk Pariwisata" in analysis_data:
        for item in analysis_data["Ide Monetisasi & Produk Pariwisata"]:
            story.append(Paragraph(f"👉 {item.get('poin', '')}", point_style))
            story.append(Paragraph(item.get('deskripsi', ''), description_style))
    story.append(Spacer(1, 0.2 * inch))

    story.append(PageBreak())

    story.append(Paragraph("Saran Peningkatan Pesan Promosi", section_title_style))
    story.append(Spacer(1, 0.1 * inch))
    if "Saran Peningkatan Pesan Promosi" in analysis_data:
        for item in analysis_data["Saran Peningkatan Pesan Promosi"]:
            story.append(Paragraph(f"👉 {item.get('poin', '')}", point_style))
            story.append(Paragraph(item.get('deskripsi', ''), description_style))
    story.append(Spacer(1, 0.2 * inch))

    story.append(Paragraph("Potensi Kolaborasi Lokal", section_title_style))
    story.append(Spacer(1, 0.1 * inch))
    if "Potensi Kolaborasi Lokal" in analysis_data:
        for item in analysis_data["Potensi Kolaborasi Lokal"]:
            story.append(Paragraph(f"👉 {item.get('poin', '')}", point_style))
            story.append(Paragraph(item.get('deskripsi', ''), description_style))
    story.append(Spacer(1, 0.2 * inch))

    story.append(Spacer(1, 0.5 * inch))
    story.append(Paragraph(f"© {datetime.now().year} Nusantara Story AI. Dibuat dengan ✨ oleh Kholish Fauzan.", footer_style))

    try:
        doc.build(story)
        buffer.seek(0)
        return buffer.getvalue()
    except Exception as e:
        print(f"Error generating analysis PDF: {e}")
        return None
"""

    print(f"\nMemproses permintaan Anda dengan IBM Granite untuk menghasilkan kode aplikasi...")

    try:
        # Panggil model IBM Granite dengan prompt yang sudah didefinisikan dalam variabel
        generated_code_output = model.invoke(my_prompt)

        print("\n--- Hasil Generasi Kode dari IBM Granite ---")
        print(generated_code_output)
        print("\n-------------------------------------------")

    except Exception as e:
        print(f"Terjadi kesalahan saat memanggil model IBM Granite: {e}")
        print("Pastikan prompt tidak terlalu panjang atau ada masalah dengan API token Replicate Anda.")

else:
    print("\nModel IBM Granite tidak dapat digunakan karena REPLICATE_API_TOKEN tidak valid atau tidak dimuat.")


--- Memulai Generasi Kode Aplikasi dengan IBM Granite ---

Memproses permintaan Anda dengan IBM Granite untuk menghasilkan kode aplikasi...

--- Hasil Generasi Kode dari IBM Granite ---
The provided code is quite comprehensive and follows good practices in terms of modularity and clarity. However, there are a few improvements that can be made to enhance readability, performance, and error handling. Here's an optimized version of the code:

1. **Consolidate Styles**: Create a centralized function to define styles to avoid repetition.
2. **Error Handling**: Improve exception handling to provide more context about the failure.
3. **Input Validation**: Add basic input validation to ensure the data structures passed to `generate_analysis_pdf` are as expected.
4. **Code Duplication**: Reduce code duplication by creating helper functions for common sections.

Here's the optimized code:

```python
from reportlab.lib.pagesizes import letter
from reportlab.platypus import SimpleDocTemplate, Par

In [ ]:
# --- Code "sidebar_content.py": Menggunakan Model untuk Generate Teks (Code Optimization) ---

if REPLICATE_API_TOKEN: # Pastikan token sudah dimuat sebelum mencoba menggunakan model
    print("\n--- Memulai Generasi Kode Aplikasi dengan IBM Granite ---")
    my_prompt = """
Review the following Python Streamlit component code for custom sidebar functionality. Suggest any improvements related to structure, best practices, UI/UX enhancement, or performance. If no issues are found, confirm it's clean. Then, generate an improved version based on your suggestions.

```python
import streamlit as st

def render_custom_sidebar_content():
    st.markdown("---")
    st.subheader("Jelajahi Halaman Lain")
    st.page_link("app.py", label="Beranda Utama", icon="🏠")
    st.page_link("pages/2_Panduan & Tips.py", label="Panduan & Tips", icon="💡")
    st.page_link("pages/3_Contoh & Inspirasi.py", label="Contoh & Inspirasi", icon="✨")
    st.page_link("pages/4_Tentang Saya.py", label="Tentang Saya", icon="👤")

    st.markdown("---")

    st.subheader("Bagaimana Kami Membantu Anda?")
    st.markdown(\"""
    Nusantara Story adalah teman setia Anda dalam **mengungkap dan membagikan pesona narasi Indonesia**. Kami memandu Anda melalui empat langkah mudah:

    * **1. Masukkan Detail Objek**: Mulai dengan memberikan informasi kunci tentang objek budaya atau destinasi wisata Anda. Semakin detail, semakin kaya hasilnya!
    * **2. Rangkai Kisah Otentik**: Biarkan Kami menyusun narasi yang indah, memukau, dan relevan dengan esensi cerita Anda. ✨
    * **3. Analisis Potensi Promosi**: Dapatkan wawasan cerdas tentang strategi promosi yang efektif dan ide pengembangan ekonomi lokal yang inovatif. 📈
    * **4. Unduh & Bagikan**: Kisah dan analisis Anda siap untuk disebarluaskan, menginspirasi, dan menarik perhatian dunia! 📊
    \""")

    st.info("💡 **Tips Cepat:** Semakin detail input Anda, semakin berkualitas hasil narasi dan analisis dari Kami! Ayo berikan informasi selengkapnya.")

def render_sidebar_expander_content():
    with st.expander("Tentang Aplikasi Ini"):
        st.markdown(\"""
        **Nusantara Story** adalah proyek inovatif yang memanfaatkan teknologi AI Gemini untuk **menggali dan mempromosikan kekayaan budaya serta potensi pariwisata Indonesia**.

        Kami juga menggunakan **IBM Granite** untuk optimasi kode aplikasi ini. Dedikasi kami adalah menciptakan solusi yang intuitif dan efektif demi kemajuan narasi lokal.
        \""")

---
Next, my app.py is like this:
from utils.sidebar_content import render_custom_sidebar_content, render_sidebar_expander_content

Please refine this based on your code generation later to import in on app.py
"""
    print(f"\nMemproses permintaan Anda dengan IBM Granite untuk menghasilkan kode aplikasi...")

    try:
        # Panggil model IBM Granite dengan prompt yang sudah didefinisikan dalam variabel
        generated_code_output = model.invoke(my_prompt)

        print("\n--- Hasil Generasi Kode dari IBM Granite ---")
        print(generated_code_output)
        print("\n-------------------------------------------")

    except Exception as e:
        print(f"Terjadi kesalahan saat memanggil model IBM Granite: {e}")
        print("Pastikan prompt tidak terlalu panjang atau ada masalah dengan API token Replicate Anda.")

else:
    print("\nModel IBM Granite tidak dapat digunakan karena REPLICATE_API_TOKEN tidak valid atau tidak dimuat.")


--- Memulai Generasi Kode Aplikasi dengan IBM Granite ---

Memproses permintaan Anda dengan IBM Granite untuk menghasilkan kode aplikasi...

--- Hasil Generasi Kode dari IBM Granite ---
The provided Streamlit code for custom sidebar functionality looks clean and follows good practices. It uses Streamlit components effectively and maintains a clear structure. Here are a few suggestions for improvement:

1. **Modularize Content**: To enhance readability and maintainability, you could separate the content into smaller functions or classes. This will make it easier to update or reuse parts of the sidebar without affecting other parts.

2. **Use Streamlit's Markdown Component**: Instead of using `st.markdown("---")` multiple times, consider using Streamlit's `st.markdown` function with a newline (`\n`) to create separate sections.

3. **Improve Expandable Content**: In the `render_sidebar_expander_content` function, you can make use of Streamlit's `st.caption` for a cleaner look when the e

In [ ]:
# --- Code "style.css": Menggunakan Model untuk Generate Teks (Code Optimization) ---

if REPLICATE_API_TOKEN: # Pastikan token sudah dimuat sebelum mencoba menggunakan model
    print("\n--- Memulai Generasi Kode Aplikasi dengan IBM Granite ---")
    my_prompt = """
Review the following custom CSS used to style a Streamlit web app. Provide a cleaned-up and optimized version of this CSS if necessary. Focus on any redundant selectors or better scoping strategies.

```css
div[data-testid="stTextInput"],
div[data-testid="stSelectbox"],
div[data-testid="stTextArea"],
div[data-testid="stDateInput"],
div[data-testid="stNumberInput"] {
    margin-bottom: 15px;
}

div[data-testid="stTextInput"] input,
div[data-testid="stTextArea"] textarea,
div[data-testid="stNumberInput"] input,
div[data-testid="stDateInput"] input {
    border-radius: 0.5rem;
    border: 1px solid #BDC3C7;
    padding: 0.75rem 1rem;
    font-size: 1rem;
    color: #333;
    width: 100%;
    box-sizing: border-box;
    transition: border-color 0.3s ease, box-shadow 0.3s ease;
}

div[data-testid="stTextInput"] input:focus,
div[data-testid="stTextArea"] textarea:focus,
div[data-testid="stNumberInput"] input:focus,
div[data-testid="stDateInput"] input:focus {
    border-color: #1ABC9C;
    box-shadow: 0 0 0 0.2rem rgba(26, 188, 156, 0.25);
    outline: none;
}

div[data-testid="stSelectbox"] .st-emotion-cache-1gcsa8n {
    border-radius: 0.5rem;
    border: 1px solid #BDC3C7;
    background-color: white;
    box-sizing: border-box;
}
div[data-testid="stSelectbox"] .st-emotion-cache-1gcsa8n:focus-within {
    border-color: #1ABC9C;
    box-shadow: 0 0 0 0.2rem rgba(26, 188, 156, 0.25);
    outline: none;
}
div[data-testid="stSelectbox"] .st-emotion-cache-1gcsa8n .st-emotion-cache-1d37g3y {
    color: #333;
    font-size: 1rem;
    padding: 0.75rem 1rem;
}

.st-emotion-cache-10q20q p {
    font-weight: 600;
    color: #555555;
    margin-bottom: 5px;
}
```
"""
    print(f"\nMemproses permintaan Anda dengan IBM Granite untuk menghasilkan kode aplikasi...")

    try:
        # Panggil model IBM Granite dengan prompt yang sudah didefinisikan dalam variabel
        generated_code_output = model.invoke(my_prompt)

        print("\n--- Hasil Generasi Kode dari IBM Granite ---")
        print(generated_code_output)
        print("\n-------------------------------------------")

    except Exception as e:
        print(f"Terjadi kesalahan saat memanggil model IBM Granite: {e}")
        print("Pastikan prompt tidak terlalu panjang atau ada masalah dengan API token Replicate Anda.")

else:
    print("\nModel IBM Granite tidak dapat digunakan karena REPLICATE_API_TOKEN tidak valid atau tidak dimuat.")


--- Memulai Generasi Kode Aplikasi dengan IBM Granite ---

Memproses permintaan Anda dengan IBM Granite untuk menghasilkan kode aplikasi...

--- Hasil Generasi Kode dari IBM Granite ---

The provided CSS styles are specific to Streamlit widgets and seem well-scoped to their respective elements. However, to ensure optimal performance and readability, we can slightly refactor it:

1. Combine similar selectors to avoid redundancy.
2. Use a more generic class for common styles, reducing the number of selectors.

Here's the cleaned-up and optimized CSS:

```css
.st-input-container {
    margin-bottom: 15px;
}

.st-input-container input,
.st-input-container textarea {
    border-radius: 0.5rem;
    border: 1px solid #BDC3C7;
    padding: 0.75rem 1rem;
    font-size: 1rem;
    color: #333;
    width: 100%;
    box-sizing: border-box;
    transition: border-color 0.3s ease, box-shadow 0.3s ease;
}

.st-input-container input:focus,
.st-input-container textarea:focus {
    border-color: #1ABC9C